In [ ]:
import os
import json
from tqdm import tqdm

# Directory paths
json_dir = "Pavement-datasets/rdd2022/United_States/ann"       # Replace with the path to your JSON files
output_dir = "Pavement-datasets/rdd2022/United_States/labels"     # Replace with the desired output directory

# Class mapping: Map 'classTitle' to numerical class IDs
class_mapping = {
    "longitudinal crack": 0,
    "transverse crack": 1,
    "alligator crack": 2,
    "pothole": 3,
    "block crack": 4,
    "repair" : 5,
    "other corruption" : 6
}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through JSON files
for json_file in tqdm(os.listdir(json_dir)):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_dir, json_file)
        
        # Load JSON data
        with open(json_path, "r") as f:
            data = json.load(f)
        
        # Get image dimensions
        img_width = data["size"]["width"]
        img_height = data["size"]["height"]
        
        # Prepare output file
        output_txt_path = os.path.join(output_dir, json_file.replace(".json", ".txt"))
        
        with open(output_txt_path, "w") as txt_file:
            # Process each object in the JSON
            for obj in data.get("objects", []):
                class_name = obj.get("classTitle")
                if class_name not in class_mapping:
                    continue  # Skip if class is not mapped
                
                class_id = class_mapping[class_name]
                
                # Extract bounding box coordinates
                points = obj["points"]["exterior"]
                x_min, y_min = points[0]
                x_max, y_max = points[1]
                
                # Calculate YOLO format values
                x_center = ((x_min + x_max) / 2) / img_width
                y_center = ((y_min + y_max) / 2) / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height
                
                # Ensure values are within [0,1] after normalization
                x_center = max(min(x_center, 1.0), 0.0)
                y_center = max(min(y_center, 1.0), 0.0)
                width = max(min(width, 1.0), 0.0)
                height = max(min(height, 1.0), 0.0)
                
                # Write to TXT
                txt_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
